
# Wav2Vec2를 사용한 음성 인식

wav2vec 2.0의 사전 학습된 모델을 사용하여 음성 인식을 수행하는 방법

## 개요

음성 인식 프로세스는 다음과 같습니다.

1. 오디오 파형에서 음향 특징 추출 .
2. 딥러닝 모델을 이용하여 음향 특징의 프레임별로 음성 클래스 분류  
3. 클래스 확률 시퀀스에서 가설 생성 --> 예측된 소리를 연결해서 실제 단어로 만드는 과정 (CTC 디코딩 기법)




## 파이프라인 만들기

먼저, 특징 추출과 분류를 수행하는 Wav2Vec2 모델을 만듭니다.

torchaudio에서 사용할 수 있는 Wav2Vec2 사전 학습된 가중치에는 두 가지 유형이 있습니다. ASR(Automatic Speech Recognition) 작업에 맞게 미세 조정된 가중치와 미세 조정되지 않은 가중치입니다.

Wav2Vec2(및 HuBERT) 모델은 자체 감독 방식으로 학습됩니다. 먼저 표현 학습을 위해 오디오로만 학습한 다음 추가 레이블이 있는 특정 작업에 맞게 미세 조정합니다.

번들 객체는 모델 및 기타 정보를 인스턴스화하는 인터페이스를 제공합니다.



모델은 다음과 같이 구성할 수 있습니다. 이 프로세스는 사전 훈련된 가중치를 자동으로 가져와 모델에 로드합니다.




## 데이터 로딩

Creative Commos BY 4.0 라이선스를 받은 [VOiCES 데이터 세트](https://iqtlabs.github.io/voices/)_의 음성 데이터를 사용합니다.




데이터를 로드하려면 `torchaudio.load` 함수를 사용합니다.

샘플링 속도가 파이프라인에서 기대하는 것과 다르면, `torchaudio.functional.resample` 함수를 사용하여 리샘플링을 할 수 있습니다.




## 음성 특징 추출

오디오에서 음향 특징을 추출



반환된 특징(features)은 텐서들의 리스트입니다. 각 텐서는 트랜스포머 레이어의 출력입니다.

- 첫 번째 차원: 배치 크기 (batch size).
1은 이 텐서가 한 번에 하나의 음성 데이터(또는 입력)를 처리했음을 의미  
- 두 번째 차원: 시퀀스 길이 또는 프레임 수
Transformer 모델에서 시퀀스의 각 시간 단계(프레임)에 대한 특징을 추출한 것. 예를 들어 169는 음성 신호가 169개의 프레임으로 나뉘어 각각에 대해 특징이 계산된 것을 의미.  
- 세 번째 차원: 특징 차원 (feature dimension)  추출된 특징 벡터의 크기.  임베딩 차원.



## 특징 분류

음향 특징이 추출되면, 다음 단계는 이를 여러 카테고리로 분류하는 것입니다.

Wav2Vec2 모델은 특징 추출과 분류를 한 단계에서 수행하는 메서드를 제공합니다.




출력은 로짓(logit) 형태로 제공됩니다.

1: 배치 크기 (한 번에 처리된 입력 음성 데이터의 개수)  
169: 음성 신호가 나누어진 프레임의 개수 (시간적 시퀀스 길이)  
29: 각 프레임에 대해 예측할 수 있는 클래스(음소 또는 문자)의 개수  

모델이 169개의 프레임에 대해 각각 29개의 클래스 중 하나를 예측하는 logit 또는 분류 결과를 생성했다는 것을 의미  




우리는 시간 축을 따라 특정 라벨에 대한 강한 신호가 나타나는 것을 볼 수 있습니다.




## 전사(Transcription) 생성

전사(transcript)는 음성 데이터를 텍스트로 변환한 결과를 의미합니다.

라벨 확률의 시퀀스로부터 이제 전사를 생성하려고 합니다. 가설을 생성하는 과정은 종종 "디코딩"이라고 불립니다.

디코딩은 단순한 분류보다 더 정교한 작업입니다. 왜냐하면 특정 시간 단계에서의 디코딩이 주변 관찰에 영향을 받을 수 있기 때문입니다.

예를 들어, "night"와 "knight"라는 단어를 생각해보면, 일반적인 대화에서 "night"가 "knight"보다 훨씬 자주 나타나지만, "knight with a sword"와 같은 문맥에서 정확하게 "knight"를 전사하려면 디코딩 과정에서 충분한 문맥을 볼 때까지 최종 결정을 미뤄야 합니다.

디코딩 기술은 여러 가지가 있으며, 이들은 외부 자원(예: 단어 사전과 언어 모델)을 필요로 합니다.

여기서는 단순화를 위해 외부 구성 요소에 의존하지 않고, 각 시간 단계에서 가장 좋은 가설을 선택하는 그리디(greedy) 디코딩을 수행할 것입니다. 따라서 문맥 정보는 사용되지 않으며, 하나의 전사만 생성될 수 있습니다.

먼저 그리디 디코딩 알고리즘을 정의하는 것부터 시작하겠습니다.


이제 디코더 객체를 생성하고 label을 디코딩합니다.

결과를 확인하고 오디오를 다시 들어보겠습니다.


